In [ ]:
import pandas as pd
import seaborn as sns
import json
import janitor
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
pd. set_option('display.max_columns', None)
pd. set_option('display.max_rows', 20)
sns.set(rc={'figure.figsize':(15,10)})
import warnings
warnings.filterwarnings("ignore")

# module_path = os.path.abspath(os.path.join(''))
# if module_path not in sys.path:
#     sys.path.append(module_path+"/python_sql/")
    
from datetime import datetime,timedelta
# import matplotlib.pyplot as plt
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 30)
# sns.set(rc={'figure.figsize':(9,8)})
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
# pip install pyjanitor

In [ ]:
# pip install dataprep

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt
plt.style.use('ggplot') # makes plot looks much nicer

In [ ]:
from dataprep.eda import plot
from dataprep.eda import create_report

## Data Loading

In [ ]:
from connect_sql import ConnectSQL
connect_obj = ConnectSQL()
vc = connect_obj.query_db("""select * from vc_purchase_settled; """)

In [ ]:
# # Load VC Txn Data
# vc = pd.read_csv('C:/Users/abhishek/Downloads/vc_fraud_01-09.csv')

In [ ]:
vc.head(2)

In [ ]:
# vc.loan_id.value_counts()

In [ ]:
vc.shape

In [ ]:
vc=vc.drop('mcc',axis=1)

In [ ]:
# From String To Dict Conversion
vc['msg_dict'] = vc['message'].apply(lambda x: json.loads(x))

In [ ]:
vc['msg_dict'].iloc[0]

In [ ]:
## Unmask the msg_dict Column
exp_vc = pd.concat([vc.drop('msg_dict', axis=1), pd.DataFrame(vc['msg_dict'].tolist())], axis=1)

In [ ]:
exp_vc.columns

In [ ]:
select_cols = ['id','updated_at', 'eventCode', 'eventType', 'amount',
        'timestamp', 'transactionCode','activityType', 'mcc', 'merchantName', 'merchantNumber','network', 'merchantLocation', 'msgEventId']
df = exp_vc[select_cols]

In [ ]:
df.head(2)

In [ ]:
# df[df['loan_id']=='885465']

In [ ]:
# df.loan_id.isna().sum()

In [ ]:
df.shape

## MCC -> Category JSON Loading

In [ ]:
category_df = pd.read_json('mcc_codes.json')

In [ ]:
category_df.head()

In [ ]:
category_df.shape, category_df['mcc'].nunique(), category_df['irs_description'].nunique()

In [ ]:
df['mcc'].nunique()

# MCC -> Category Mapping Dictionay Creation

In [ ]:
# Creating Category -> Mcc Mapping 
mcc_cat_dict = {str(mcc) : category for mcc, category in zip(category_df['mcc'], category_df['irs_description'])}

In [ ]:
df['category'] = [mcc_cat_dict.get(str(m), 'Others') for m in df['mcc']]

In [ ]:
# category_df[category_df['mcc']==5964].values

In [ ]:
df.head(5)

## Category Distribution

In [ ]:
# 405

In [ ]:
# plot(df_bad, "bad_flag_7dpd_1quin")

In [ ]:
# Top 10 merchant category Distribution
m = sns.countplot(y="category", data=df, order=pd.value_counts(df['category']).iloc[:10].index)
m.set( xlabel = "Count", ylabel = "Merchant Category", title="Top 10 Merchant Category Distribution")

In [ ]:
## Top 10 Mcc Code Distribution
c = sns.countplot(y="mcc", data=df, order=pd.value_counts(df['mcc']).iloc[:10].index)
c.set( xlabel = "Count", ylabel = "MCC Code", title="Top 10 MCC Code Distribution")

In [ ]:
## Top 10 Merchant Name Distribution
d = sns.countplot(y="merchantName", data=df, order=pd.value_counts(df['merchantName']).iloc[:30].index)
d.set( xlabel = "Count", ylabel = "Merchant", title="Top 30 Merchants")

## Merchants Distribution

In [ ]:
# df['category'].value_counts()[:10].plot(kind='barh')

In [ ]:
plot(df, "merchantName")

In [ ]:
df['merchantName'].value_counts()[:30].plot(kind='barh') ## 

In [ ]:
df[df['merchantName']=='PAYPAL4'].head(5)

In [ ]:
df[df['merchantName'].isin(['MERCADO PAGO 1', 'MERCADO PAGO 4', 'SNP*MERCADOPAGO'])].head(5)  ## Merchant Cleaning Required

In [ ]:
df.columns

## Location Distribution

In [ ]:
plot(df, "merchantLocation")

In [ ]:
# plot(df, "amount")

## MCC Distribution

In [ ]:
plot(df, "mcc")

In [ ]:
category_df['mcc'].nunique()

In [ ]:
## Txnn AMount vs Category Wise

In [ ]:
df = df[~df['category'].isnull()]
test = df.groupby(['category', 'mcc']).size().reset_index(name='count').drop(0).sort_values(by='count', ascending=True)

In [ ]:
test.to_csv('merchant_category_count.csv', index=False)

In [ ]:
df[['mcc', 'category', 'merchantName']].to_csv('vc_merchant_data.csv', index=False)

In [ ]:
category_df.rename({'irs_description': 'category'}, inplace=True)
category_df[['mcc', 'irs_description']].to_csv('mcc_category_mapping.csv', index=False)

## High Frequency MCC, Category, Merchant Name & Location

In [ ]:
hf_sample = df[df['mcc'].isin(['5964', '4816', '5399'])][['mcc', 'merchantName', 'category', 'merchantLocation']]
sample_report = create_report(hf_sample, title='VC HF Report')

In [ ]:
sample_report

# deliquency

In [ ]:
# m_name=pd.read_csv("e:/data/merchant_name_20220907.csv")

In [ ]:
# loan_id=pd.read_csv("e:/data/vps_id_loan_id_20220907.csv").drop(columns='customer_id')

# df=pd.merge(df,loan_id,on='id',how='inner')

In [ ]:
from connect_sql import ConnectSQL
connect_obj = ConnectSQL()
loan_id = connect_obj.query_db("""
select vps.id,vc.loan_id,l.total_price,l.customer_id  from vc_purchase_settled vps 
left join vc_balance vcb on cast(vps.balance_id as int)=vcb.id
left join vw_vc_accounts_info vc on vc.id=vcb.loan_card_id
left join loan l on vc.loan_id = l.id; """)

loan_id.head()

In [ ]:
df=pd.merge(df,loan_id,on='id',how='inner')

In [ ]:
df=df[['merchantName','mcc','category','loan_id','total_price']]

In [ ]:
df=df.drop_duplicates()

In [ ]:
bad_flag=pd.read_feather("loan_level_data2022-10-26.ft")
bad_flag.head()

In [ ]:
df_bad=pd.merge(df,bad_flag,on='loan_id',how='inner')

In [ ]:
df_bad.head()

In [ ]:
df_bad[df_bad['mcc']==9222]

In [ ]:
df_bad.loan_id.value_counts()

In [ ]:
df_bad['bad_flag_7dpd_1quin'].mean()

In [ ]:
df_bad['bad_flag_15dpd_1quins'].mean()

In [ ]:
mcc_bad_rate=df_bad.groupby('mcc').agg({'bad_flag_7dpd_1quin':'mean','bad_flag_15dpd_1quins':'mean','loan_id':'count','total_price':'sum'}).reset_index()
mcc_bad_rate.rename(columns = {'loan_id':'total_count','total_price':'gmv'}, inplace = True)
mcc_bad_rate

In [ ]:
7801
7800
9406
9222

In [ ]:
df_bad[df_bad['mcc']==9222]

In [ ]:
mcc_bad_rate[mcc_bad_rate['mcc']==9222]

In [ ]:
mcc_bad_rate1=mcc_bad_rate[mcc_bad_rate['total_count']>5]
mcc_bad_rate1

In [ ]:
mcc_bad_rate2=pd.merge(mcc_bad_rate1,df_bad[['mcc','category']],on='mcc',how='inner').drop_duplicates(subset=['mcc']).sort_values(by=['bad_flag_7dpd_1quin'],ascending=False)

In [ ]:
mcc_bad_rate2.head(20)

In [ ]:
mcc_bad_rate2.to_csv('top_20_bad_mcc.csv',index=False)

In [ ]:
# mcc_bad_rate1.sort_values()

In [ ]:
bad_rate=(df_bad.groupby('loan_start_date')
        .agg(bad_flag=('bad_flag_7dpd_1quin','mean'))
        .reset_index())

import plotly.express as px
px.line(bad_rate,x='loan_start_date',y='bad_flag')

### VC_category_bad_rate

In [ ]:
category=(df_bad.groupby('category').agg(total_amount=('total_price','sum'),
                                  average_amount=('total_price','mean'),
                                  bad_flag=('bad_flag_7dpd_1quin','mean'),
                                  no_of_unique_customers=('customer_id','count'))
                            .reset_index()
                            )

sns.barplot( data=category.sort_values(by='average_amount',ascending=False).iloc[:30],y="category",x='average_amount')
plt.show()

In [ ]:
m=sns.barplot( data=category.sort_values(by='bad_flag',ascending=False).iloc[:30],y="category",x='bad_flag')
plt.xticks(rotation=90)
plt.show()

In [ ]:
category.sort_values(by='no_of_unique_customers',ascending=False).head(10)

In [ ]:
category.to_csv("category_wise_bad_rate_2022_10_26.csv",index=0)

### VC_merchant_bad_rate

In [ ]:
merchant=(df_bad.groupby('merchantName').agg(total_amount_in_vc_loans=('total_price','sum'),
                                  average_amount_in_vc_loans=('total_price','mean'),
                                  bad_flag=('bad_flag_7dpd_1quin','mean'),
                                  no_of_unique_customers=('customer_id','count'))
                                .reset_index()
                                .sort_values(by='bad_flag',ascending=False).iloc[:50])

m=sns.barplot( data=merchant,y="merchantName",x='bad_flag')
plt.show()

In [ ]:
merchant.sort_values(by='no_of_unique_customers',ascending=False).head(20)

In [ ]:
merchant.to_csv("merchant_wise_bad_rate_2022_10_26.csv",index=0)